# **imports and variables:**

In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

!pip install plotly
import plotly.graph_objects as go

In [2]:
sp100 = [
    "AAPL", "ABBV", "ABT", "ACN", "ADBE", "AIG", "AMD", "AMGN", "AMT", "AMZN",
    "ANET", "APA", "APD", "AVGO", "AXP", "BA", "BAC", "BIIB", "BK", "BKNG",
    "BLK", "BMY", "BSX", "C", "CAT", "CHTR", "CL", "CMCSA", "COF", "COP",
    "COST", "CRM", "CSCO", "CVS", "CVX", "DHR", "DIS", "DOW", "DUK", "EMR",
    "EXC", "F", "FDX", "GD", "GE", "GILD", "GM", "GOOG", "GOOGL", "GS",
    "HON", "IBM", "INTC", "JNJ", "JPM", "KO", "LIN", "LLY", "LMT", "LOW",
    "MA", "MAR", "MCD", "MDLZ", "MDT", "MET", "MMM", "MO", "MRK", "MS",
    "MSFT", "NEE", "NFLX", "NKE", "NVDA", "ORCL", "PEP", "PFE", "PG", "PM",
    "PYPL", "QCOM", "RTX", "SBUX", "SCHW", "SO", "SPG", "T", "TGT", "TMO",
    "TMUS", "TSLA", "TXN", "UNH", "UNP", "UPS", "USB", "V", "VZ", "WBA",
    "WFC", "WMT", "XOM"
]

# **Functions:**

In [3]:
#takes a df and creates new columns that checks if the current 'High' price is higher than certain SMAs
def sma_filtering(df):
  filt20 = df["SMA20"] < df["High"]
  filt50 = df["SMA50"] < df["High"]
  filt100 = df["SMA100"] < df["High"]
  df["SMA20_Above"] = filt20
  df["SMA50_Above"] = filt50
  df["SMA100_Above"] = filt100
  df["above_avg"] = filt20 & filt50 & filt100
  return df

In [4]:
#takes as an input a stock name and period and returns the stock's df.
def stock_df(stock, period):
  dat = yf.Ticker(stock)
  df = dat.history(period=period).reset_index()
  df = df.loc[:, ['Date', 'Open', 'High', 'Low', 'Close']]
  return df

In [5]:
# takes a given stock and iterates over 'bunch-size" stocks at a time,
# applying 'func' to each bunch independantly
def iter(stock, bunch_size, func=None):
  i=0

  #taking the stock and converting it to a df.
  if type(stock) == str:
    cur_stock_df = stock_df(stock, '1y')
  elif type(stock) == pd.DataFrame:
    cur_stock_df = stock
  else:
    return 'not a valid dataFrame'

  #iterating while the bunch size equals the 'cur_bunch'
  #if they are not equal, it means we are on the last portion and we need to stop after this one.
  while True:
    cur_bunch = cur_stock_df.iloc[i:i+bunch_size, :]
    if len(cur_bunch) < bunch_size:
      break
    #if the user entered a function, apply it for each bunch.
    if func==bunch_mean:
      func(cur_stock_df, cur_bunch, bunch_size, i+bunch_size-1)
    elif func==weighted_bunch_mean:
      func(cur_stock_df, cur_bunch, bunch_size, i+bunch_size-1)
    i+=1
  return cur_stock_df

In [6]:
def bunch_mean(df, bunch, bunch_size, new_col_index):
  #creates a new column if not already exists.
  new_col = f"SMA{bunch_size}"
  if new_col not in df.columns:
        df[new_col] = None

  #calculates the mean of the current bunch.
  mean = bunch['High'].mean()

  #inputs the calculated mean in the corresponding column.
  df.loc[ new_col_index, f"SMA{bunch_size}"] = mean

In [7]:
def weighted_bunch_mean(df, bunch, bunch_size, new_col_index):
  weighted_mean=0
  index_sum=0
  new_col = f"WMA{bunch_size}"
  if new_col not in df.columns:
        df[new_col] = None

  #calculates the mean of the current bunch.
  for index, row in bunch.iterrows():
    index_sum += (index+1)
    weighted_mean += (index+1) * row['High']


  #inputs the calculated mean in the corresponding column.
  df.loc[ new_col_index, f"WMA{bunch_size}"] = (weighted_mean / index_sum)


In [8]:
def exponential_moving_average(df, n):
    # computes the exponential moving average (EMA) over n days
    alpha = 2 / (n + 1)
    ema_values = []       # list to store computed EMA values

    for i, price in enumerate(df["Close"]):
        if i == 0:
            ema_values.append(price)   # first EMA equals first price
        else:
            prev_ema = ema_values[-1]  # previous EMA
            ema_values.append(alpha * price + (1 - alpha) * prev_ema)  # EMA formula

    df[f"EMA_{n}"] = ema_values   # create new EMA column
    return df


In [9]:
# calculates WMA for a given window size using your weighted_bunch_mean function
def calc_wma_column(df, size):
    col = f"WMA{size}"
    df[col] = None

    for i in range(len(df)):
        # not enough rows to calculate WMA
        if i + 1 < size:
            continue

        # take the last 'size' rows
        bunch = df.iloc[i + 1 - size : i + 1]

        # use your function to place the result
        weighted_bunch_mean(df, bunch, size, i)

    return col

# Hull Moving Average
def hull_moving_average(df, n):
    half = int(n / 2)
    sqrt_n = int(np.sqrt(n))

    # calculate WMA(n)
    col_full = calc_wma_column(df, n)

    # calculate WMA(n/2)
    col_half = calc_wma_column(df, half)

    # create temporary values: 2*WMA(n/2) - WMA(n)
    df["HMA_temp"] = 2 * df[col_half] - df[col_full]

    # calculate WMA on the temporary values using sqrt(n)
    out_col = f"HMA{n}"
    df[out_col] = None

    for i in range(len(df)):
        # not enough rows for sqrt(n)
        if i + 1 < sqrt_n:
            continue

        # take the last sqrt(n) values
        bunch = df["HMA_temp"].iloc[i + 1 - sqrt_n : i + 1]

        # skip if there are missing values
        if bunch.isna().any():
            continue

        # compute WMA (same style as your function)
        weighted_mean = 0
        index_sum = 0

        for idx, val in enumerate(bunch):
            weight = idx + 1
            weighted_mean += weight * val
            index_sum += weight

        df.loc[i, out_col] = weighted_mean / index_sum

    return df


In [10]:
#finds the highest high price in the last n days for each row
def highest_high(df, window):
    df = df.copy()
    df[f'HH{window}'] = df['High'].rolling(window=window).max()
    return df

In [11]:
#finds the lowest low price in the last n days for each row
def lowest_low(df, window):
    df = df.copy()
    df[f'LL{window}'] = df['Low'].rolling(window=window).min()
    return df

In [12]:
def calculate_adx(df, n=14):

    df = df.copy()

    # Step 1: Price differences
    df['H_diff'] = df['High'].diff()
    df['L_diff'] = -df['Low'].diff()

    df['DM+'] = np.where((df['H_diff'] > df['L_diff']) & (df['H_diff'] > 0), df['H_diff'], 0)
    df['DM-'] = np.where((df['L_diff'] > df['H_diff']) & (df['L_diff'] > 0), df['L_diff'], 0)

    # Step 2: True Range
    df['TR1'] = df['High'] - df['Low']
    df['TR2'] = (df['High'] - df['Close'].shift()).abs()
    df['TR3'] = (df['Low'] - df['Close'].shift()).abs()
    df['TR'] = df[['TR1', 'TR2', 'TR3']].max(axis=1)

    # Step 3: Wilder smoothing
    df['TR_smooth'] = df['TR'].ewm(alpha=1/n, adjust=False).mean()
    df['DM+_smooth'] = df['DM+'].ewm(alpha=1/n, adjust=False).mean()
    df['DM-_smooth'] = df['DM-'].ewm(alpha=1/n, adjust=False).mean()

    # Step 4: DI+
    df['DI+'] = 100 * (df['DM+_smooth'] / df['TR_smooth'])
    df['DI-'] = 100 * (df['DM-_smooth'] / df['TR_smooth'])

    # Step 5: DX
    df['DX'] = 100 * ( (df['DI+'] - df['DI-']).abs() / (df['DI+'] + df['DI-']) )

    # Step 6: ADX
    df['ADX'] = df['DX'].ewm(alpha=1/n, adjust=False).mean()

    return df


# **Code:**

In [13]:
top10 = []
top10_dfs = []
i=0

for stock in sp100:
  if len(top10) == 10:
    break

  df = iter(stock, 20, bunch_mean)
  df = iter(df, 50, bunch_mean)
  df = iter(df, 100, bunch_mean)
  sma_filtering(df)

  df = iter(df, 20, weighted_bunch_mean)
  df = iter(df, 50, weighted_bunch_mean)
  df = iter(df, 100, weighted_bunch_mean)

  df = exponential_moving_average(df, 20)
  df = exponential_moving_average(df, 50)

  df = hull_moving_average(df, 20)
  df = hull_moving_average(df, 50)

  df = highest_high(df,20)
  df = lowest_low(df,20)

  df = calculate_adx(df, n=14)

  if(df.iloc[-1,-1]):
    top10.append(stock)
    top10_dfs.append(df)
  i+=1

top10

['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'AMD', 'AMGN', 'AMT', 'AMZN']

In [14]:
top10_dfs[0]

,Date,Open,High,Low,Close,SMA20,SMA50,SMA100,SMA20_Above,SMA50_Above,...,TR2,TR3,TR,TR_smooth,DM+_smooth,DM-_smooth,DI+,DI-,DX,ADX
0,2025-02-26 00:00:00-05:00,243.269729,243.916902,238.092297,239.316956,None,None,None,False,False,...,NaN,NaN,5.824605,5.824605,0.000000,0.000000,0.000000,0.000000,NaN,NaN
1,2025-02-27 00:00:00-05:00,238.371077,241.407845,236.031269,236.270233,None,None,None,False,False,...,2.090889,3.285686,5.376576,5.792603,0.000000,0.147216,0.000000,2.541454,100.000000,100.000000
2,2025-02-28 00:00:00-05:00,235.921733,241.039427,229.201025,240.790512,None,None,None,False,False,...,4.769194,7.069208,11.838402,6.224445,0.000000,0.624575,0.000000,10.034234,100.000000,100.000000
3,2025-03-03 00:00:00-05:00,240.740733,242.971018,235.085389,236.997055,None,None,None,False,False,...,2.180506,5.705123,7.885629,6.343101,0.137971,0.579963,2.175131,9.143207,61.564482,97.254606
4,2025-03-04 00:00:00-05:00,236.678463,239.028223,233.661598,234.906174,None,None,None,False,False,...,2.031168,3.335457,5.366625,6.273353,0.128116,0.640236,2.042220,10.205647,66.651821,95.068693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2026-02-20 00:00:00-05:00,258.970001,264.750000,258.160004,264.579987,268.068043,268.680741,268.4312,False,False,...,4.170013,2.419983,6.589996,6.507022,1.474034,1.958968,22.652970,30.105446,14.125663,19.744228
248,2026-02-23 00:00:00-05:00,263.489990,269.429993,263.380005,266.179993,269.0807,268.473977,268.580351,True,True,...,4.850006,1.199982,6.049988,6.474377,1.703031,1.819042,26.304166,28.096013,3.293826,18.569200
249,2026-02-24 00:00:00-05:00,267.859985,274.890015,267.709991,272.140015,270.009193,268.382008,268.77492,True,True,...,8.710022,1.529999,8.710022,6.634066,1.971387,1.689110,29.716126,25.461162,7.711442,17.793645
250,2026-02-25 00:00:00-05:00,271.779999,274.940002,271.049988,274.230011,270.670938,268.294236,268.941343,True,True,...,2.799988,1.090027,3.890015,6.438062,1.834145,1.568459,28.489078,24.362292,7.808286,17.080405


In [15]:
#check if the code works correctly
df_copy = df.copy()
df_copy = df_copy.head(40)
df_copy = highest_high(df_copy, 20)
df_copy = lowest_low(df_copy, 20)
print(df_copy)


                        Date        Open        High         Low       Close  \
0  2025-02-26 00:00:00-05:00  214.940002  218.160004  213.089996  214.350006   
1  2025-02-27 00:00:00-05:00  218.350006  219.970001  208.369995  208.740005   
2  2025-02-28 00:00:00-05:00  208.649994  212.619995  206.990005  212.279999   
3  2025-03-03 00:00:00-05:00  213.350006  214.009995  202.550003  205.020004   
4  2025-03-04 00:00:00-05:00  200.110001  206.800003  197.429993  203.800003   
5  2025-03-05 00:00:00-05:00  204.800003  209.979996  203.259995  208.360001   
6  2025-03-06 00:00:00-05:00  204.399994  205.770004  198.300003  200.699997   
7  2025-03-07 00:00:00-05:00  199.490005  202.270004  192.529999  199.250000   
8  2025-03-10 00:00:00-04:00  195.600006  196.729996  190.850006  194.539993   
9  2025-03-11 00:00:00-04:00  193.899994  200.179993  193.399994  196.589996   
10 2025-03-12 00:00:00-04:00  200.720001  201.520004  195.289993  198.889999   
11 2025-03-13 00:00:00-04:00  198.169998

In [16]:
i = 0
for df in top10_dfs:
    df['Date'] = pd.to_datetime(df['Date'])

    fig = go.Figure(data=go.Scatter(x=df['Date'], y=df['High'], mode='lines', name='Price'))
    fig.add_scatter(x=df['Date'], y=df['SMA20'], mode='lines', name='SMA 20')
    fig.add_scatter(x=df['Date'], y=df['SMA50'], mode='lines', name='SMA 50')
    fig.add_scatter(x=df['Date'], y=df['SMA100'], mode='lines', name='SMA 100')


    high_above_sma = df['High'].where(df['above_avg'], other=np.nan)

    fig.add_scatter(
        x=df['Date'],
        y=high_above_sma,
        mode='lines',
        line=dict(color='white', width=3, dash='dot'),
        name='Above all SMAs'
    )

    fig.update_layout(
        title=f"{top10[i]} Stock",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_dark"
    )
    fig.update_xaxes(tickformat="%m-%Y")
    fig.show()
    i += 1


In [17]:
i = 0
for df in top10_dfs:

    df['Date'] = pd.to_datetime(df['Date'])

    fig = go.Figure(data=go.Scatter(x=df['Date'], y=df['High'], mode='lines', name='Price', line=dict(color='white')))

    fig.add_scatter(x=df['Date'], y=df['SMA20'], mode='lines', name='SMA 20', line=dict(color='green'))
    fig.add_scatter(x=df['Date'], y=df['WMA20'], mode='lines', name='WMA 20', line=dict(color='blue'))
    fig.add_scatter(x=df['Date'], y=df['EMA_20'], mode='lines', name='EMA 20', line=dict(color='red'))
    fig.add_scatter(x=df['Date'], y=df['HMA20'], mode='lines', name='HMA 20', line=dict(color='cyan'))

    fig.update_layout(
        title=f"{top10[i]} Stock",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_dark"
    )

    fig.update_xaxes(tickformat="%m-%Y")
    fig.show()

    i += 1


In [18]:
i = 0
for df in top10_dfs:

    df['Date'] = pd.to_datetime(df['Date'])

    fig = go.Figure(data=go.Scatter(x=df['Date'], y=df['High'], mode='lines', name='Price', line=dict(color='white')))

    fig.add_scatter(x=df['Date'], y=df['SMA50'], mode='lines', name='SMA 50', line=dict(color='green'))
    fig.add_scatter(x=df['Date'], y=df['WMA50'], mode='lines', name='WMA 50', line=dict(color='blue'))
    fig.add_scatter(x=df['Date'], y=df['EMA_50'], mode='lines', name='EMA 50', line=dict(color='red'))
    fig.add_scatter(x=df['Date'], y=df['HMA50'], mode='lines', name='HMA 50', line=dict(color='cyan'))

    fig.update_layout(
        title=f"{top10[i]} Stock",
        xaxis_title="Date",
        yaxis_title="Price",
        template="plotly_dark"
    )

    fig.update_xaxes(tickformat="%m-%Y")
    fig.show()

    i += 1


In [19]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

i = 0
for df in top10_dfs:

    df['Date'] = pd.to_datetime(df['Date'])
    df = df.replace({None: np.nan})  # make comparisons safe

    # price above ALL moving averages
    above_20 = (df['High'] > df['SMA20']) & (df['High'] > df['WMA20']) & (df['High'] > df['EMA_20']) & (df['High'] > df['HMA20'])
    high_above_20 = df['High'].where(above_20, other=np.nan)

    above_50 = (df['High'] > df['SMA50']) & (df['High'] > df['WMA50']) & (df['High'] > df['EMA_50']) & (df['High'] > df['HMA50'])
    high_above_50 = df['High'].where(above_50, other=np.nan)

    fig = make_subplots(rows=1, cols=2, subplot_titles=("Window 20", "Window 50"))

    # ================= Window 20 =================
    fig.add_trace(go.Scatter(x=df['Date'], y=df['High'], mode='lines', name='Price', line=dict(color='white')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA20'], mode='lines', name='SMA 20', line=dict(color='green')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['WMA20'], mode='lines', name='WMA 20', line=dict(color='blue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA_20'], mode='lines', name='EMA 20', line=dict(color='red')), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['HMA20'], mode='lines', name='HMA 20', line=dict(color='cyan')), row=1, col=1)

    # dashed white line
    fig.add_scatter(x=df['Date'], y=high_above_20, mode='lines',
                    line=dict(color='orange', width=3, dash='dot'),
                    name='Above all MAs 20', row=1, col=1)

    # ================= Window 50 =================
    fig.add_trace(go.Scatter(x=df['Date'], y=df['High'], mode='lines', name='Price', line=dict(color='white')), row=1, col=2)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['SMA50'], mode='lines', name='SMA 50', line=dict(color='green')), row=1, col=2)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['WMA50'], mode='lines', name='WMA 50', line=dict(color='blue')), row=1, col=2)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['EMA_50'], mode='lines', name='EMA 50', line=dict(color='red')), row=1, col=2)
    fig.add_trace(go.Scatter(x=df['Date'], y=df['HMA50'], mode='lines', name='HMA 50', line=dict(color='cyan')), row=1, col=2)

    # dashed white line
    fig.add_scatter(x=df['Date'], y=high_above_50, mode='lines',
                    line=dict(color='orange', width=3, dash='dot'),
                    name='Above all MAs 50', row=1, col=2)

    fig.update_layout(title=f"{top10[i]} — Comparison: Window 20 vs Window 50", template="plotly_dark")
    fig.update_xaxes(tickformat="%m-%Y")

    fig.show()
    i += 1


/tmp/ipython-input-167/911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-167/911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-167/911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-167/911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-167/911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-167/911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-167/911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-167/911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-167/911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



/tmp/ipython-input-167/911485289.py:9: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [20]:
i = 0
for df in top10_dfs:

    df['Date'] = pd.to_datetime(df['Date'])

    fig = go.Figure()

    fig.add_scatter(
        x=df['Date'],
        y=df['ADX'],
        mode='lines',
        name='ADX 14',
        line=dict(color='red')
    )

    fig.update_layout(
        title=f"{top10[i]} — ADX Indicator",
        xaxis_title="Date",
        yaxis_title="ADX Value",
        template="plotly_dark"
    )

    fig.update_xaxes(tickformat="%m-%Y")
    fig.show()

    i += 1


In [31]:
import pandas as pd
import numpy as np

def generate_strategy_table(df):
    epsilon = 0.001
    # Selecting the last 100 records
    subset = df.tail(100).copy().reset_index(drop=True)

    rows = []
    current_hold = 0 # Initial holding state (x = 0)

    for idx in range(len(subset)):
        p = subset.loc[idx, 'High']
        s = subset.loc[idx, 'SMA20']

        # Decision Logic (f) based on x (current_hold) and p/s
        if current_hold == 0:
            if p > (s + epsilon):
                f = 1 # Buy action
            else:
                f = 0 # Stay out
        else: # current_hold == 1
            if p < (s + epsilon):
                f = 0 # Sell action
            else:
                f = 1 # Keep holding

        # Create row with 'Day' (i) starting from 1
        rows.append({
            'Day': idx + 1,
            'Price': round(p, 3),
            'Hold': current_hold,
            'SMA20': round(s, 3),
            'Strategy_Output': f
        })

        # Update holding state for next day
        current_hold = f

    # Convert to DataFrame and set 'Day' as index to remove default 0-99 index
    res_df = pd.DataFrame(rows).set_index('Day')
    return res_df

# --- Execution ---

# 1. Prepare data
aapl_df = stock_df("AAPL", "1y")
aapl_df = iter(aapl_df, 20, bunch_mean)

# 2. Generate the cleaned table
strategy_table = generate_strategy_table(aapl_df)

# 3. Display
print(strategy_table)

       Price  Hold    SMA20  Strategy_Output
Day                                         
1    258.747     0  247.276                1
2    258.577     1  248.220                1
3    256.910     1  249.149                1
4    258.028     1  250.452                1
5    257.509     1  251.827                1
..       ...   ...      ...              ...
96   264.750     0  268.068                0
97   269.430     0  269.081                1
98   274.890     1  270.009                1
99   274.940     1  270.671                1
100  276.110     1  271.546                1

[100 rows x 4 columns]


In [32]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

strategy_table

,Price,Hold,SMA20,Strategy_Output
Day,,,,
1,258.747,0,247.276,1
2,258.577,1,248.220,1
3,256.910,1,249.149,1
4,258.028,1,250.452,1
5,257.509,1,251.827,1
...,...,...,...,...
96,264.750,0,268.068,0
97,269.430,0,269.081,1
98,274.890,1,270.009,1


In [33]:
import pandas as pd
import numpy as np

def generate_strategy_table(df):
    epsilon = 0.001
    # Selecting the last 100 records
    subset = df.tail(100).copy().reset_index(drop=True)

    rows = []
    current_hold = 0 # Initial holding state (x = 0)
    buy_price = 0.0  # Track entry price
    a = 0.0          # Last closed profit

    for idx in range(len(subset)):
        p = subset.loc[idx, 'High']
        s = subset.loc[idx, 'SMA20']

        # Decision Logic (f) based on x (current_hold) and p/s
        if current_hold == 0:
            if p > (s + epsilon):
                f = 1 # Buy action
            else:
                f = 0 # Stay out
        else: # current_hold == 1
            if p < (s + epsilon):
                f = 0 # Sell action
            else:
                f = 1 # Keep holding

        # Profit Logic
        if current_hold == 0 and f == 1:
            buy_price = p
            profit = 0.0
        elif f == 1:
            profit = p - buy_price
        elif current_hold == 1 and f == 0:
            profit = p - buy_price
            a = profit
        else:
            profit = a

        # Create row with 'Day' (i) starting from 1
        rows.append({
            'Day': idx + 1,
            'Price': round(p, 3),
            'Hold': current_hold,
            'SMA20': round(s, 3),
            'Strategy_Output': f,
            'Profit': round(profit, 3)
        })

        # Update holding state for next day
        current_hold = f

    # Convert to DataFrame and set 'Day' as index to remove default 0-99 index
    res_df = pd.DataFrame(rows).set_index('Day')
    return res_df


#prepare data
aapl_df = stock_df("AAPL", "1y")
aapl_df = iter(aapl_df, 20, bunch_mean)
strategy_table = generate_strategy_table(aapl_df)
print(strategy_table)

       Price  Hold    SMA20  Strategy_Output  Profit
Day                                                 
1    258.747     0  247.276                1   0.000
2    258.577     1  248.220                1  -0.170
3    256.910     1  249.149                1  -1.836
4    258.028     1  250.452                1  -0.719
5    257.509     1  251.827                1  -1.238
..       ...   ...      ...              ...     ...
96   264.750     0  268.068                0  -1.810
97   269.430     0  269.081                1   0.000
98   274.890     1  270.009                1   5.460
99   274.940     1  270.671                1   5.510
100  276.110     1  271.546                1   6.680

[100 rows x 5 columns]


In [34]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

strategy_table

,Price,Hold,SMA20,Strategy_Output,Profit
Day,,,,,
1,258.747,0,247.276,1,0.000
2,258.577,1,248.220,1,-0.170
3,256.910,1,249.149,1,-1.836
4,258.028,1,250.452,1,-0.719
5,257.509,1,251.827,1,-1.238
...,...,...,...,...,...
96,264.750,0,268.068,0,-1.810
97,269.430,0,269.081,1,0.000
98,274.890,1,270.009,1,5.460
